## 1. 환경 설정

`(1) Env 환경변수`

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [3]:
import os
from glob import glob

from pprint import pprint
import json

## 2. 벡터 저장소 (Vector Store)

### 2.1 Chroma

- 사용자 편의성이 우수한 오픈소스 벡터 저장소
 - `langchain-chroma` 패키지 설치

`(1) 벡터 저장소 초기화`

In [ ]:
# 벡터 저장소에 문서를 저장할 때 적용할 임베딩 모델
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")


# 벡터 저장소 생성
from langchain_chroma import Chroma

chroma_db = Chroma(
    collection_name="ai_sample_collection",
    embedding_function=embeddings_model,
    persist_directory="./chroma_db",
)


In [6]:
chroma_db.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

`(2) 벡터 저장소 관리`  

- 문서 추가: `vector_store.add_documents(documents, ids)`

In [8]:
from langchain_core.documents import Document

# 문서 컬렉션
documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

# Document 객체 생성
doc_objects = []
# enumerate()를 사용하여 documents 리스트를 순회하면서 인덱스와 내용을 함께 가져옴
# start=1로 설정하여 Chapter 번호를 1부터 시작하도록 함
for i, content in enumerate(documents, start=1):
    doc = Document(
        page_content=content,
        metadata={"source": "AI_textbook", "chapter": f"Chapter {i}"},
    )
    doc_objects.append(doc)


# 순차적 ID 리스트 생성
doc_ids = [f"DOC_{i}" for i in range(1, len(doc_objects) + 1)]

# 문서를 벡터 저장소에 저장
# 벡터 저장소에 문서를 추가하는 메서드
# documents: Document 객체 리스트 - 추가할 문서들의 리스트 
# ids: 문자열 리스트 - 각 문서에 할당할 고유 ID 리스트
# 반환값: 추가된 문서들의 ID 리스트
added_doc_ids = chroma_db.add_documents(documents=doc_objects, ids=doc_ids)

# 벡터 저장소에 저장된 문서를 확인
print(f"{len(added_doc_ids)}개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.")
print(added_doc_ids)

chroma_db.get()

5개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.
['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4', 'DOC_5']


{'ids': ['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4', 'DOC_5'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야입니다.',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.',
  '컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 1', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 5', 'source': 'AI_textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [9]:
# 저장된 문서 검색
query = "인공지능과 머신러닝의 관계는?"
# 코사인 유사도를 사용하여 쿼리와 가장 유사한 문서 2개를 검색
# 코사인 유사도는 두 벡터 간의 각도를 기반으로 유사도를 측정
# k=2는 상위 2개의 가장 유사한 문서를 반환

# 코사인 유사도는 두 벡터 간의 각도의 코사인 값을 이용하여 유사도를 측정하는 방법입니다.
# 두 벡터가 같은 방향을 가리키면 코사인 값이 1이 되고, 수직이면 0, 반대 방향이면 -1이 됩니다.
# 즉, 코사인 유사도가 1에 가까울수록 두 벡터(문서)가 유사하다는 의미입니다.

# 코사인 유사도는 벡터의 방향 유사도를 측정하는 지표입니다.
# 두 벡터가 이루는 각도의 코사인 값을 계산하여 -1에서 1 사이의 값을 가집니다.
# 1에 가까울수록 방향이 유사하고, -1에 가까울수록 반대 방향임을 의미합니다.
results = chroma_db.similarity_search(query, k=2)

print(f"\n쿼리: {query}")
print("가장 유사한 문서:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")


쿼리: 인공지능과 머신러닝의 관계는?
가장 유사한 문서:
- 머신러닝은 인공지능의 하위 분야입니다. [출처: AI_textbook, Chapter 2]
- 딥러닝은 머신러닝의 한 종류입니다. [출처: AI_textbook, Chapter 3]


In [10]:
# 현재 저장된 컬렉션 데이터 확인
chroma_db.get()

{'ids': ['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4', 'DOC_5'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야입니다.',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.',
  '컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 1', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 5', 'source': 'AI_textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

- 문서 수정: `vector_store.update_document(document_id, document)`

In [11]:
# 업데이트할 문서 생성
updated_document_1 = Document(
    page_content="인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.",
    metadata={"source": "AI_textbook", "chapter": "Chapter 1"},
)

updated_document_2 = Document(
    page_content="머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다.",
    metadata={"source": "AI_textbook", "chapter": "Chapter 2"},
)

updated_document_3 = Document(
    page_content="딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.",
    metadata={"source": "AI_textbook", "chapter": "Chapter 3"},
)


# 단일 문서 업데이트
# update_document() 메서드를 사용하여 단일 문서를 업데이트합니다.
# 파라미터:
#   - document_id: 업데이트할 문서의 고유 ID (str)
#   - document: 업데이트할 새로운 Document 객체
#     - page_content: 문서의 내용
#     - metadata: 문서의 메타데이터 (source, chapter 등)
# 반환값: 없음
chroma_db.update_document(document_id="DOC_1", document=updated_document_1)

# 여러 문서 한 번에 업데이트
chroma_db.update_documents(
    ids=["DOC_2", "DOC_3"],
    documents=[updated_document_2, updated_document_3]
)

print("문서 업데이트 완료")

문서 업데이트 완료


In [12]:
# 저장된 문서 검색 예시
query = "인공지능과 머신러닝의 관계는?"
results = chroma_db.similarity_search(query, k=2)

print(f"\n쿼리: {query}")
print("가장 유사한 문서:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")


쿼리: 인공지능과 머신러닝의 관계는?
가장 유사한 문서:
- 머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다. [출처: AI_textbook, Chapter 2]
- 인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다. [출처: AI_textbook, Chapter 1]


- 문서 삭제: `vector_store.delete(ids)`

In [13]:
chroma_db.delete(ids=["DOC_5"])

In [14]:
# 컬렉션 확인
chroma_db.get()

{'ids': ['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.',
  '머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 1', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

`(3) 문서 검색`  

- 유사도 검색
    - 주어진 쿼리와 가장 유사한 문서를 반환
    -  k=2는 상위 2개의 결과를 반환하도록 지정
    - filter를 사용하여 특정 출처의 문서만 검색 가능

In [15]:
query = "인공지능과 머신러닝의 차이점은 무엇인가요?"
results = chroma_db.similarity_search(
    query,
    k=2,
    # filter 파라미터:
    # - 검색 결과를 필터링하는 조건을 지정
    # - 메타데이터의 특정 필드를 기준으로 문서를 필터링
    # - 예: source가 "AI_textbook"인 문서만 검색
    # - 필수값 여부: 선택 사항 (Optional)
    filter={"source": "AI_textbook"}
)

print("유사도 검색 결과:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")

유사도 검색 결과:
- 머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다. [출처: AI_textbook, Chapter 2]
- 인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다. [출처: AI_textbook, Chapter 1]


- 유사도 점수가 포함된 검색
    - 유사도 점수를 함께 반환
    - 점수가 낮을수록 더 유사한 것을 의미 (거리 기준으로 점수가 산정되기 때문)

In [16]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
# similarity_search_with_score() 함수
# - 용도: 주어진 쿼리와 가장 유사한 문서를 검색하고, 각 문서의 유사도 점수를 함께 반환
# - 파라미터:
#   - query: 검색할 텍스트 쿼리 (필수)
#   - k: 반환할 결과의 개수 (선택, 기본값=4)
#   - filter: 메타데이터 기반 필터링 조건 (선택)
# - 반환값: (Document, float) 튜플의 리스트
#   - Document: 검색된 문서 객체
#   - float: 유사도 점수 (낮을수록 더 유사함)
results = chroma_db.similarity_search_with_score(
    query,
    k=2,
    filter={"source": "AI_textbook"}
)

print("점수가 포함된 유사도 검색 결과:\n")
for doc, score in results:
    print(f"- 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")
    print()

점수가 포함된 유사도 검색 결과:

- 점수: 0.5772
  내용: 딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.
  [출처: AI_textbook, Chapter 3]

- 점수: 0.7292
  내용: 인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.
  [출처: AI_textbook, Chapter 1]



- 관련성 점수가 포함된 검색
    - 문서와 함께 0에서 1 사이의 관련성 점수를 반환
    - 0은 가장 관련성이 낮고, 1은 가장 관련성이 높음을 의미

In [17]:
query = "딥러닝은 어떤 분야에서 사용되나요?"

# similarity_search_with_relevance_scores() 함수
# - 용도: 주어진 쿼리와 가장 유사한 문서를 검색하고, 0~1 사이의 관련성 점수를 함께 반환
# - 파라미터:
#   - query: 검색할 텍스트 쿼리 (필수)
#   - k: 반환할 결과의 개수 (선택, 기본값=4)
#   - filter: 메타데이터 기반 필터링 조건 (선택)
# - 반환값: (Document, float) 튜플의 리스트
#   - Document: 검색된 문서 객체
#   - float: 관련성 점수 (1에 가까울수록 더 관련성이 높음)

# similarity_search_with_score()는 벡터 간 거리(L2/유클리디안)를 점수로 사용
# similarity_search_with_relevance_scores()는 코사인 유사도를 0~1로 정규화한 점수 사용
results = chroma_db.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI_textbook"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")
    print()

쿼리: 딥러닝은 어떤 분야에서 사용되나요?

검색 결과 (관련성 점수 포함):
- 관련성 점수: 0.5919
  내용: 딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.
  [출처: AI_textbook, Chapter 3]

- 관련성 점수: 0.4844
  내용: 인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.
  [출처: AI_textbook, Chapter 1]



`(4) 벡터 저장소 로드`  

In [20]:
chroma_db2 = Chroma(
    collection_name="ai_sample_collection",
    embedding_function=embeddings_model,
    persist_directory="./chroma_db",
)

chroma_db2.get()


{'ids': ['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.',
  '머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 1', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [21]:
# 미리 임베딩된 쿼리 벡터를 사용하여 검색
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = chroma_db2.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI_textbook"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")
    print()

쿼리: 딥러닝은 어떤 분야에서 사용되나요?

검색 결과 (관련성 점수 포함):
- 관련성 점수: 0.5919
  내용: 딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.
  [출처: AI_textbook, Chapter 3]

- 관련성 점수: 0.4844
  내용: 인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.
  [출처: AI_textbook, Chapter 1]



### 2.2 FAISS(Facebook AI Similarity Search)

- 효율적인 벡터 유사도 검색 및 클러스터링을 위한 오픈소스 벡터 저장소 
- `faiss-cpu` 패키지 설치

`(1) 벡터 저장소 초기화`

In [22]:
# 벡터 저장소에 문서를 저장할 때 적용할 임베딩 모델
# langchain_huggingface.embeddings: HuggingFace의 임베딩 모델을 LangChain에서 사용할 수 있게 해주는 패키지
# HuggingFaceEmbeddings: 텍스트를 벡터로 변환하는 임베딩 모델 클래스
from langchain_huggingface.embeddings import HuggingFaceEmbeddings


embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")  # bge-m3 모델 사용

# faiss: Facebook AI가 개발한 효율적인 벡터 유사도 검색 라이브러리
import faiss

# langchain_community.docstore.in_memory: 문서를 메모리에 저장하는 문서 저장소
from langchain_community.docstore.in_memory import InMemoryDocstore

# langchain_community.vectorstores: 다양한 벡터 저장소를 제공하는 패키지
# FAISS: Facebook의 벡터 검색 라이브러리를 LangChain에서 사용할 수 있게 래핑한 클래스
from langchain_community.vectorstores import FAISS

# FAISS 인덱스 초기화 (유클리드 거리 사용)
# FAISS 벡터 저장소를 위한 인덱스를 초기화합니다.
# - IndexFlatL2: L2(유클리드) 거리를 사용하는 기본 인덱스 타입
#   - L2 거리: 두 벡터 간의 직선 거리를 계산하는 방식
# - "hello world"는 임베딩 모델의 출력 차원을 알아내기 위한 샘플 텍스트
#   - 임베딩 모델은 어떤 입력이든 동일한 차원(1024)의 벡터를 출력
#   - 실제 내용은 중요하지 않음, 차원 수만 필요
faiss_index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world")))
print("FAISS 인덱스 초기화 완료")

FAISS 인덱스 초기화 완료


In [23]:
# FAISS 벡터 저장소의 벡터 차원 수 (임베딩 차원 수)
# FAISS 벡터 저장소의 벡터 차원 수를 확인합니다.
# - faiss_index.d: FAISS 인덱스의 벡터 차원 수를 반환하는 속성
# - bge-m3 모델은 1024 차원의 벡터를 생성합니다.
faiss_index.d

1024

In [24]:
# FAISS 벡터 저장소 생성
# FAISS 클래스의 주요 파라미터:
# - embedding_function: 텍스트를 벡터로 변환하는 임베딩 모델
#   - HuggingFaceEmbeddings 모델(bge-m3)을 사용하여 1024차원 벡터 생성
# - index: FAISS 인덱스 객체 
#   - IndexFlatL2를 사용하여 L2(유클리드) 거리 기반 유사도 검색
# - docstore: 문서 저장소
#   - InMemoryDocstore()를 사용하여 문서를 메모리에 저장
# - index_to_docstore_id: 인덱스 ID와 문서 ID 간의 매핑
#   - 빈 딕셔너리로 시작하고 문서 추가시 자동으로 매핑 생성
faiss_db = FAISS(
    embedding_function=embeddings_model,  # 텍스트 임베딩 모델
    index=faiss_index,                   # FAISS 인덱스
    docstore=InMemoryDocstore(),         # 메모리 기반 문서 저장소
    # docstore=DocstoreExplorer("./docstore"),  # 디스크 기반 문서 저장소
    index_to_docstore_id={},             # 인덱스-문서 ID 매핑
)

# 저장된 문서의 갯수 확인
# index.ntotal: FAISS 인덱스에 저장된 전체 벡터의 수를 반환
faiss_db.index.ntotal

0

`(2) 벡터 저장소 관리`  

- 문서 추가: `vector_store.add_documents(documents, ids)`

In [25]:

# langchain_core.documents에서 Document 클래스를 임포트합니다.
# Document 클래스는 텍스트 콘텐츠와 메타데이터를 포함하는 문서 객체를 생성하는데 사용됩니다.
from langchain_core.documents import Document

# 인공지능 관련 텍스트로 구성된 문서 컬렉션을 리스트로 정의합니다.
# 각 문자열은 인공지능의 다양한 분야에 대한 설명을 담고 있습니다.
documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.", 
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

# Document 객체의 리스트를 생성합니다.
# enumerate를 사용하여 각 문서에 1부터 시작하는 인덱스를 부여합니다.
doc_objects = []
for i, content in enumerate(documents, start=1):
    # 각 문서마다 Document 객체를 생성합니다.
    # - page_content: 실제 문서 내용
    # - metadata: 문서의 출처(AI_textbook)와 장 번호(Chapter N) 정보
    doc = Document(
        page_content=content,
        metadata={"source": "AI_textbook", "chapter": f"Chapter {i}"},
    )
    doc_objects.append(doc)

# 문서 식별을 위한 고유 ID 리스트를 생성합니다.
# 'DOC_1'부터 'DOC_5'까지의 ID를 생성합니다.
doc_ids = [f"DOC_{i}" for i in range(1, len(doc_objects) + 1)]

# 생성된 Document 객체들을 FAISS 벡터 저장소에 추가합니다.
# add_documents 메서드는 문서와 해당 ID를 받아 벡터화하여 저장합니다.
added_doc_ids = faiss_db.add_documents(documents=doc_objects, ids=doc_ids)

# 벡터 저장소에 추가된 문서의 수와 ID 목록을 출력합니다.
print(f"{len(added_doc_ids)}개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.")
print(added_doc_ids)

5개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.
['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4', 'DOC_5']


In [26]:
# 저장된 문서의 갯수 확인
# faiss_db.index.ntotal: FAISS 벡터 저장소에 저장된 총 벡터(문서)의 수를 반환
# ntotal 속성은 현재 인덱스에 포함된 벡터의 총 개수를 나타냄
faiss_db.index.ntotal

5

- 문서 삭제: `vector_store.delete(ids)`

In [27]:
faiss_db.delete(ids=["DOC_5"])

True

In [28]:
# 컬렉션 확인
faiss_db.index.ntotal

4

`(3) 문서 검색`  

- 유사도 검색
    - 주어진 쿼리와 가장 유사한 문서를 반환
    - k=2는 상위 2개의 결과를 반환하도록 지정
    - filter를 사용하여 특정 출처의 문서만 검색 가능

In [29]:
query = "인공지능과 머신러닝의 차이점은 무엇인가요?"
results = faiss_db.similarity_search(
    query,
    k=2,
    filter={"source": "AI_textbook"}
)

print("유사도 검색 결과:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")

유사도 검색 결과:
- 머신러닝은 인공지능의 하위 분야입니다. [출처: AI_textbook, Chapter 2]
- 딥러닝은 머신러닝의 한 종류입니다. [출처: AI_textbook, Chapter 3]


- 유사도 점수가 포함된 검색
    - 유사도 점수를 함께 반환
    - 점수가 낮을수록 더 유사한 것을 의미

In [30]:
query = "딥러닝은 어떤 분야에서 사용되나요?"

# similarity_search_with_score() 함수는 similarity_search()와 유사하지만 
# 검색 결과와 함께 유사도 점수를 반환한다는 점이 다릅니다.
# similarity_search()는 문서만 반환하는 반면,
# similarity_search_with_score()는 (문서, 점수) 튜플의 리스트를 반환합니다.
# 점수는 L2 거리를 나타내며 값이 작을수록 더 유사함을 의미합니다.
results = faiss_db.similarity_search_with_score(
    query,
    k=2,
    filter={"source": "AI_textbook"}
)

print("점수가 포함된 유사도 검색 결과:\n")
for doc, score in results:
    print(f"- 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")
    print()

점수가 포함된 유사도 검색 결과:

- 점수: 0.6517
  내용: 딥러닝은 머신러닝의 한 종류입니다.
  [출처: AI_textbook, Chapter 3]

- 점수: 0.8442
  내용: 머신러닝은 인공지능의 하위 분야입니다.
  [출처: AI_textbook, Chapter 2]



- 관련성 점수가 포함된 검색
    - 문서와 함께 0에서 1 사이의 관련성 점수를 반환
    - 0은 가장 관련성이 낮고, 1은 가장 관련성이 높음을 의미

In [31]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
# similarity_search_with_relevance_scores() 함수는 검색 결과와 함께 0~1 사이의 관련성 점수를 반환합니다.
# similarity_search_with_score()는 L2 거리 기반 점수(낮을수록 유사)를 반환하는 반면,
# 이 함수는 코사인 유사도 기반의 정규화된 점수(높을수록 유사)를 반환합니다.
#
# 파라미터:
# - query: 검색할 쿼리 문자열
# - k: 반환할 최대 문서 수 (기본값: 4)
# - filter: 검색 결과를 필터링할 조건 (예: {"source": "AI_textbook"})
# 코사인 유사도는 두 벡터 간의 각도를 기반으로 유사도를 측정합니다.
# - 값의 범위: -1 ~ 1 (1에 가까울수록 유사)
# - 방향의 유사성을 측정하며 크기는 고려하지 않음
# - 정규화된 벡터에서 잘 동작

# L2 거리(유클리디안 거리)는 두 벡터 간의 실제 거리를 계산합니다.
# - 값의 범위: 0 ~ ∞ (0에 가까울수록 유사) 
# - 벡터의 크기와 방향 모두 고려
# - 스케일에 민감하므로 정규화가 중요

# similarity_search_with_relevance_scores는 코사인 유사도 기반으로
# 0~1 사이로 정규화된 점수를 반환합니다 (1이 가장 유사)
results = faiss_db.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI_textbook"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")
    print()

쿼리: 딥러닝은 어떤 분야에서 사용되나요?

검색 결과 (관련성 점수 포함):
- 관련성 점수: 0.5392
  내용: 딥러닝은 머신러닝의 한 종류입니다.
  [출처: AI_textbook, Chapter 3]

- 관련성 점수: 0.4031
  내용: 머신러닝은 인공지능의 하위 분야입니다.
  [출처: AI_textbook, Chapter 2]



`(4) 로컬에 저장 및 로드`  

In [32]:
# 로컬에 저장
faiss_db.save_local("faiss_ai_samaple_index")

In [33]:
# 로컬에 저장된 FAISS 벡터 저장소 불러오기
# FAISS.load_local() 함수를 사용하여 로컬에 저장된 벡터 저장소를 불러옵니다.
# 
# 파라미터:
# - folder_path: 벡터 저장소가 저장된 로컬 폴더 경로 (예: "faiss_ai_samaple_index")
# - embeddings: 임베딩 모델 객체 (텍스트를 벡터로 변환하는데 사용)
# - allow_dangerous_deserialization: 직렬화된 객체의 안전하지 않은 역직렬화를 허용할지 여부 (기본값: False)
#   - True로 설정하면 보안상 위험할 수 있으나, 저장된 모든 객체를 완전히 복원할 수 있음
#   - False로 설정하면 안전하게 역직렬화하나 일부 기능이 제한될 수 있음
#
# 반환값:
# - FAISS: 로드된 FAISS 벡터 저장소 객체

faiss_db2 = FAISS.load_local(
    # allow_dangerous_deserialization=True 설정은 보안상 위험할 수 있습니다.
    # - 악의적인 코드가 포함된 직렬화된 객체를 역직렬화할 경우 시스템이 손상될 수 있음
    # - 신뢰할 수 있는 소스의 데이터만 역직렬화하는 것이 안전
    # - 개발 환경에서만 사용하고 프로덕션 환경에서는 False로 설정 권장
    "faiss_ai_samaple_index", embeddings_model, allow_dangerous_deserialization=True
)

In [34]:
# 저장된 문서 객체를 확인
# 저장된 문서 객체를 딕셔너리 형태로 확인
# docstore._dict는 내부 구현에 의존적이므로 권장되지 않음
faiss_db2.docstore._dict

{'DOC_1': Document(metadata={'source': 'AI_textbook', 'chapter': 'Chapter 1'}, page_content='인공지능은 컴퓨터 과학의 한 분야입니다.'),
 'DOC_2': Document(metadata={'source': 'AI_textbook', 'chapter': 'Chapter 2'}, page_content='머신러닝은 인공지능의 하위 분야입니다.'),
 'DOC_3': Document(metadata={'source': 'AI_textbook', 'chapter': 'Chapter 3'}, page_content='딥러닝은 머신러닝의 한 종류입니다.'),
 'DOC_4': Document(metadata={'source': 'AI_textbook', 'chapter': 'Chapter 4'}, page_content='자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.')}

In [35]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = faiss_db2.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI_textbook"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")
    print()

쿼리: 딥러닝은 어떤 분야에서 사용되나요?

검색 결과 (관련성 점수 포함):
- 관련성 점수: 0.5392
  내용: 딥러닝은 머신러닝의 한 종류입니다.
  [출처: AI_textbook, Chapter 3]

- 관련성 점수: 0.4031
  내용: 머신러닝은 인공지능의 하위 분야입니다.
  [출처: AI_textbook, Chapter 2]



## 3. RAG 검색기



### 3.1 Semantic Search(의미론적 검색) 
- Vector Store 검색기 사용

`(1) 벡터 저장소 초기화`
- cosine distance 기준으로 인덱싱 

In [36]:
# 문서 로딩을 위한 TextLoader 임포트
from langchain_community.document_loaders import TextLoader  # 텍스트 파일을 로드하기 위한 도구

# 텍스트 분할을 위한 CharacterTextSplitter 임포트 
from langchain_text_splitters import CharacterTextSplitter  # 텍스트를 청크 단위로 분할하는 도구

# Hugging Face 토크나이저 임포트
from transformers import AutoTokenizer  # 텍스트를 토큰화하기 위한 Hugging Face의 자동 토크나이저


# 데이터 로드
def load_text_files(txt_files):
    data = []

    for text_file in txt_files:
        loader = TextLoader(text_file, encoding='utf-8')
        data += loader.load()

    return data

korean_txt_files = glob(os.path.join('data', '*_KR.txt')) 
korean_data = load_text_files(korean_txt_files)


# Hugging Face의 임베딩 모델이 사용한 토크나이저 지정 
# AutoTokenizer.from_pretrained() 함수:
# - 목적: Hugging Face의 사전 학습된 토크나이저를 로드
# - 인자: 
#   - "BAAI/bge-m3": 사용할 토크나이저 모델의 이름
# - 반환: 토크나이저 객체
# - 기능: 텍스트를 토큰화하여 모델이 이해할 수 있는 형태로 변환

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")
#from tiktoken import get_encoding
# tokenizer = get_encoding("cl100k_base")  # OpenAI의 gpt-4, text-embedding-ada-002 등에서 사용하는 토크나이저
# 사용 예:
# encoded = tokenizer.encode("안녕하세요")  # 텍스트를 토큰으로 인코딩
# decoded = tokenizer.decode(encoded)      # 토큰을 다시 텍스트로 디코딩
# token_count = len(encoded)              # 토큰 수 계산



# 문장을 구분하여 분할 (마침표, 느낌표, 물음표 다음에 공백이 오는 경우 문장의 끝으로 판단)
# CharacterTextSplitter를 사용하여 텍스트를 청크 단위로 분할
# 입력 파라미터:
# - tokenizer: Hugging Face 토크나이저 객체 (텍스트를 토큰화하는데 사용)
# - separator: 문장 구분자로 사용할 정규식 패턴 ([.!?] 뒤에 공백이 오는 경우)
# - chunk_size: 각 청크의 최대 토큰 수 (100개로 제한)
# - chunk_overlap: 청크 간 중복되는 토큰 수 (0으로 설정하여 중복 없음)
# - is_separator_regex: separator가 정규식 패턴임을 나타내는 플래그
# - keep_separator: 구분자를 청크에 포함할지 여부

# 사용 목적:
# 1. 긴 텍스트를 의미 있는 청크로 분할하여 벡터 저장소에 효율적으로 저장
# 2. 문장 단위로 분할하여 의미적 일관성 유지
# 3. 토큰 수 제한으로 임베딩 모델의 입력 제약 준수

text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    separator=r"[.!?]\s+",
    chunk_size=100,
    chunk_overlap=0,
    is_separator_regex=True,
    keep_separator=True,
)

korean_docs = text_splitter.split_documents(korean_data)

print("한국어 문서 수:", len(korean_docs))

한국어 문서 수: 8


In [37]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# Hugging Face의 임베딩 모델 생성
embeddings_huggingface = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# Chroma 벡터 저장소 생성하기
# Chroma 벡터 저장소 생성
# 입력 파라미터:
# - documents: 벡터화할 문서 리스트 (korean_docs)
# - embedding: 문서를 벡터화할 임베딩 모델 (HuggingFace의 bge-m3 모델 사용)
# - collection_name: 벡터 저장소의 컬렉션 이름 
# - persist_directory: 벡터 DB를 저장할 디렉토리 경로
# - collection_metadata: 벡터 유사도 계산 방식 설정
#   - hnsw:space: 'cosine' (코사인 유사도), 'l2' (유클리디안 거리), 'ip' (내적) 중 선택

# 사용 목적:
# 1. 문서를 벡터화하여 Chroma DB에 저장
# 2. 벡터 간 유사도 검색을 위한 인덱스 구축
# 3. 영구 저장소에 벡터 DB 보관

chroma_db = Chroma.from_documents(
    documents=korean_docs,
    embedding=embeddings_huggingface,    # huggingface 임베딩 사용
    collection_name="db_korean_cosine", 
    persist_directory="./chroma_db",
    collection_metadata = {'hnsw:space': 'cosine'}, # l2, ip, cosine 중에서 선택 
)

`(2) Top K`

In [38]:
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k": 2},
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_k_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

쿼리: 리비안은 언제 사업을 시작했나요?
검색 결과:
- 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다 [출처: data/리비안_KR.txt]
- .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다 [출처: data/리비안_KR.txt]


`(3) 임계값 지정`
- Similarity score threshold (기준 스코어 이상인 문서를 대상으로 추출)

In [39]:
# langchain_community.utils.math 패키지에서 cosine_similarity 함수를 임포트
# cosine_similarity: 두 벡터 간의 코사인 유사도를 계산하는 함수
# - 입력: 두 개의 벡터 또는 벡터 배열
# - 출력: -1 ~ 1 사이의 유사도 점수 (1에 가까울수록 유사)
# - 용도: 문서 검색, 텍스트 유사도 비교 등에 활용
from langchain_community.utils.math import cosine_similarity

chroma_threshold_retriever = chroma_db.as_retriever(
    search_type='similarity_score_threshold',       # 검색 방식: 유사도 점수 기반 필터링
                                                   # - 문서와 쿼리 간의 유사도를 계산하여 일정 점수 이상인 문서만 추출
                                                   # - 관련성이 높은 문서만을 선별적으로 검색하기 위해 사용
    search_kwargs={
        'score_threshold': 0.5,  # 유사도 점수 임계값 (0~1 사이)
                                # - 0.5로 설정하면 쿼리와 50% 이상 유사한 문서만 추출
                                # - 임계값이 높을수록 더 엄격한 필터링 (예: 0.8은 매우 유사한 문서만 추출)
                                # - 검색 정확도와 검색 결과 수의 균형을 맞추기 위해 적절한 임계값 설정 필요
        
        'k': 2                   # 반환할 최대 문서 개수 
                                # - 임계값을 넘는 문서들 중에서 상위 2개만 반환
                                # - 너무 많은 검색 결과를 방지하고 가장 관련성 높은 문서만 선별
                                # - 사용자에게 제공할 정보의 양을 제한하여 효율적인 검색 결과 제공
    }
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_threshold_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    #  score = cosine_similarity(
    #     [embeddings_model.embed_query(query)], 
    #     [embeddings_model.embed_query(doc.page_content)]
    #     )[0][0]
    # 쿼리와 문서 내용의 임베딩 벡터 간 코사인 유사도 계산
    # 1. 쿼리 임베딩
    query_embedding = embeddings_model.embed_query(query)
    # 2. 문서 임베딩 
    doc_embedding = embeddings_model.embed_query(doc.page_content)


    # 3. 코사인 유사도 계산 (2차원 배열로 변환 후 첫 번째 요소 추출)
    # cosine_similarity 함수는 두 벡터 간의 코사인 유사도를 계산
    # 입력 파라미터:
    # - query_embedding: 쿼리 텍스트의 임베딩 벡터 (1차원 배열)
    # - doc_embedding: 문서 텍스트의 임베딩 벡터 (1차원 배열)
    # 동작:
    # 1. [query_embedding], [doc_embedding]으로 각각을 2차원 배열로 변환 
    # 2. 두 벡터 간 코사인 유사도 계산하여 2차원 배열 [[score]] 반환
    # 3. [0][0] 인덱싱으로 실제 스코어값(float)을 추출
    # 반환값: -1 ~ 1 사이의 유사도 점수 (1에 가까울수록 유사)

    # 쿼리와 문서 간의 의미적 유사도를 수치화하여 검색 결과의 관련성을 평가
    # - 높은 유사도 점수: 쿼리의 의도/주제와 문서 내용이 잘 일치함을 의미
    # - 낮은 유사도 점수: 쿼리와 문서가 의미적으로 관련성이 낮음을 의미
    # 이를 통해 사용자 질문에 가장 적절한 문서를 선별하고 검색 품질을 정량적으로 측정 가능
    score = cosine_similarity([query_embedding], [doc_embedding])[0][0]
    
    # 결과 출력 - 문서 내용과 유사도 점수
    print(f"- {doc.page_content} [유사도: {score:.4f}]")

쿼리: 리비안은 언제 사업을 시작했나요?
검색 결과:
- 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다 [유사도: 0.6734]
- .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다 [유사도: 0.5955]


`(4) MMR(Maximal Marginal Relevance) 검색`

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

web_loader = WebBaseLoader(["https://python.langchain.com/", "https://js.langchain.com/"])

web_docs = web_loader.load()

len(web_docs)


2

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

web_loader = WebBaseLoader(["https://python.langchain.com/", "https://js.langchain.com/"])

web_docs = web_loader.load()

len(web_docs)


2

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

web_loader = WebBaseLoader(["https://python.langchain.com/", "https://js.langchain.com/"])

web_docs = web_loader.load()

len(web_docs)


2

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

web_loader = WebBaseLoader(["https://python.langchain.com/", "https://js.langchain.com/"])

web_docs = web_loader.load()

len(web_docs)


2

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

web_loader = WebBaseLoader(["https://python.langchain.com/", "https://js.langchain.com/"])

web_docs = web_loader.load()

len(web_docs)


2

In [ ]:
# MMR - 다양성 고려 (lambda_mult 작을수록 더 다양하게 추출)
chroma_mmr = chroma_db.as_retriever(
    # MMR(Maximal Marginal Relevance) 알고리즘은 검색 결과의 관련성과 다양성을 동시에 고려하는 검색 방식입니다.
    # - 관련성: 쿼리와 문서 간의 유사도
    # - 다양성: 검색된 문서들 간의 중복성을 최소화
    # 목적:
    # 1. 검색 결과의 다양성 확보
    # 2. 중복되는 정보 제거
    # 3. 포괄적인 정보 제공
    # MMR 검색은 다음과 같은 경우에는 사용하지 않는 것이 좋습니다:
    # 1. 검색 속도가 중요한 경우 - MMR은 추가 연산이 필요해 속도가 느림
    # 2. 정확도가 가장 중요한 경우 - 다양성을 위해 관련성이 다소 떨어지는 결과도 포함될 수 있음
    # 3. 검색 결과의 수가 매우 적은 경우 - 다양성을 고려할 필요가 없음
    # 4. 실시간 처리가 필요한 경우 - 계산 비용이 높아 지연이 발생할 수 있음
    search_type='mmr',
    search_kwargs={
        'k': 3,                 # 최종적으로 반환할 문서의 수

        # fetch_k는 선택적 파라미터입니다. 
        # 기본값은 k의 2배로 설정됩니다.
        # 다양성을 더 확보하고 싶다면 k보다 큰 값으로 명시적 설정이 필요합니다.
        'fetch_k': 8,           # MMR 알고리즘이 처리할 후보 문서의 수 (k보다 커야 다양성 확보 가능)
        
        'lambda_mult': 0.5,     # 관련성과 다양성의 균형을 조절하는 파라미터
                               # 1에 가까울수록 관련성 중시
                               # 0에 가까울수록 다양성 중시
                               # 0.5는 둘의 균형을 맞춤
        },
)


query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_mmr.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    score = cosine_similarity(
        [embeddings_model.embed_query(query)], 
        [embeddings_model.embed_query(doc.page_content)]
        )[0][0]
    print(f"- {doc.page_content} [유사도: {score:.4f}]")

쿼리: 리비안은 언제 사업을 시작했나요?
검색 결과:
- 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다 [유사도: 0.6734]
- .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다 [유사도: 0.5955]
- .

리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다. [유사도: 0.5690]


`(5) metadata 필터링 검색`

In [68]:
# 문서 객체의 metadata를 이용한 필터링
chrom_metadata = chroma_db.as_retriever(
    search_kwargs={
        'filter': {'source': 'data/리비안_KR.txt'},
        'k': 8, 
        }
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chrom_metadata.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

쿼리: 리비안은 언제 사업을 시작했나요?
검색 결과:
- 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다 [출처: data/리비안_KR.txt]
- .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다 [출처: data/리비안_KR.txt]
- .

리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다. [출처: data/리비안_KR.txt]
- . 리비안은 디젤 하이브리드 버전, 브라질 원메이크 시리즈를 위한 R1 GT 레이싱 버전, 4도어 세단 및 크로스오버 등 다양한 버전을 고려했습니다. 2011년에 프로토타입 해치백도 공개되었지만, R1과의 관계는 불명확합니다 [출처: data/리비안_KR.txt]


`(6) page_content 본문 필터링 검색`

In [69]:
# page_content를 이용한 필터링
chroma_content = chroma_db.as_retriever(
    search_kwargs={
        'k': 2,
        'where_document': {'$contains': '리비안'},
        }
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_content.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

쿼리: 리비안은 언제 사업을 시작했나요?
검색 결과:
- 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다 [출처: data/리비안_KR.txt]
- .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다 [출처: data/리비안_KR.txt]


### 3.2 Keyword Search(키워드 검색) 
- BM25 등

`(1) BM25 검색기 생성`

- BM25: TF-IDF (Term Frequency-Inverse Document Frequency)의 확장된 버전
- `rank_bm25` 설치

In [70]:
# 벡터 저장소에 저정한 문서 객체를 로드하여 확인
chroma_db.get().keys()

dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data', 'included'])

In [71]:
# BM25 검색기 생성을 위해 문서 객체를 로드
documents = chroma_db.get()["documents"]
metadatas = chroma_db.get()["metadatas"]

# Document 객체로 변환
from langchain_core.documents import Document
docs = [Document(page_content=content, metadata=meta) for content, meta in zip(documents, metadatas)]

print("문서의 수:" , len(docs))

# BM25 검색기 생성
from langchain_community.retrievers import BM25Retriever
bm25_retriever = BM25Retriever.from_documents(docs)

# BM25 검색기를 사용하여 검색
query = "리비안은 언제 사업을 시작했나요?"

retrieved_docs = bm25_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")


문서의 수: 8
쿼리: 리비안은 언제 사업을 시작했나요?
검색 결과:
- .

리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다. [출처: data/리비안_KR.txt]
- 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다 [출처: data/리비안_KR.txt]
- . 리비안은 디젤 하이브리드 버전, 브라질 원메이크 시리즈를 위한 R1 GT 레이싱 버전, 4도어 세단 및 크로스오버 등 다양한 버전을 고려했습니다. 2011년에 프로토타입 해치백도 공개되었지만, R1과의 관계는 불명확합니다 [출처: data/리비안_KR.txt]
- . [출처: data/테슬라_KR.txt]


In [72]:
# BM25 점수를 확인
query = "리비안은 언제 사업을 시작했나요?"
tokenized_query = query.split()
print(tokenized_query)

doc_scores = bm25_retriever.vectorizer.get_scores(tokenized_query)
doc_scores_sorted = sorted(enumerate(doc_scores), key=lambda x: x[1], reverse=True)
doc_scores_sorted

['리비안은', '언제', '사업을', '시작했나요?']


[(7, 0.5782203609632524),
 (5, 0.5616794524521536),
 (3, 0.4308683105054156),
 (0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (4, 0.0),
 (6, 0.0)]

In [76]:
# 같은 의미를 갖는 쿼리로 변경하여 대시 검색 
query = "리비안이 설립된 연도는?"

retrieved_docs = bm25_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

쿼리: 리비안이 설립된 연도는?
검색 결과:
- 테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다 [출처: data/테슬라_KR.txt]
- .

리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다. [출처: data/리비안_KR.txt]
- . [출처: data/테슬라_KR.txt]
- 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다 [출처: data/리비안_KR.txt]


In [75]:
# BM25 점수를 확인
query = "리비안이 설립된 연도는?"
tokenized_query = query.split()
print(tokenized_query)

doc_scores = bm25_retriever.vectorizer.get_scores(tokenized_query)
doc_scores_sorted = sorted(enumerate(doc_scores), key=lambda x: x[1], reverse=True)
doc_scores_sorted

['리비안이', '설립된', '연도는?']


[(1, 1.5086146557386926),
 (0, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0)]

`(2) kiwi 한국어 토크나이저`
- `kiwipiepy` 설치

In [83]:
# 한국어 토크나이저를 사용하여 문장을 토큰화하는 함수
from kiwipiepy import Kiwi

def bm25_process_func(text):
    """
    BM25Retriever에서 사용할 전처리 함수
    한국어 토크나이저를 사용하여 문장을 토큰화
    :param text: 토큰화할 문장
    :param kwii_model: Kiwi 객체 (from kiwipiepy import Kiwi 사용)
    """
    kiwi_model = Kiwi()

    kiwi_model.add_user_word('리비안', 'NNP')  # NNP: 고유명사
    kiwi_model.add_user_word('테슬라', 'NNP')  # NNP: 고유명사

    return [t.form for t in kiwi_model.tokenize(text)]


# BM25Retriever 객체 생성
bm25_retriever = BM25Retriever.from_documents(
    documents=docs,
    preprocess_func=bm25_process_func,
    )

# 이전에 사용한 검색어를 입력하여 문서를 검색
query = "리비안이 설립된 연도는?"

retrieved_docs = bm25_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

쿼리: 리비안이 설립된 연도는?
검색 결과:
- 테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다 [출처: data/테슬라_KR.txt]
- 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다 [출처: data/리비안_KR.txt]
- . 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다 [출처: data/테슬라_KR.txt]
- .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다 [출처: data/리비안_KR.txt]


In [88]:
# BM25 점수를 확인
query = "리비안이 설립된 연도는?"

kiwi_model = Kiwi()

kiwi_model.add_user_word('리비안', 'NNP')  # NNP: 고유명사
kiwi_model.add_user_word('테슬라', 'NNP')  # NNP: 고유명사

tokenized_query = [t.form for t in kiwi_model.tokenize(query)]
print(tokenized_query)

doc_scores = bm25_retriever.vectorizer.get_scores(tokenized_query)
doc_scores_sorted = sorted(enumerate(doc_scores), key=lambda x: x[1], reverse=True)
doc_scores_sorted

['리비안', '이', '설립', '되', 'ᆫ', '연도', '는', '?']


[(1, 2.3579578398183014),
 (5, 2.039726152975322),
 (4, 1.4391924308060922),
 (0, 1.1066940406176984),
 (7, 0.8168622949775345),
 (3, 0.7815426133191743),
 (2, 0.2789860806919751),
 (6, 0.0)]

### 3.3 Hybrid Search 
- 키워드 기반 검색과 시맨틱 검색을 결합하여 보다 정확하고 관련성 높은 결과를 제공하는 방법


In [80]:
from langchain.retrievers import EnsembleRetriever

ensemble_retrievers = [chroma_threshold_retriever, bm25_retriever]
ensemble_retriever = EnsembleRetriever(
    retrievers=ensemble_retrievers, 
    weights=[0.5, 0.5]
)

query = "리비안이 설립된 연도는?"

retrieved_docs = ensemble_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

쿼리: 리비안이 설립된 연도는?
검색 결과:
- 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다 [출처: data/리비안_KR.txt]
- .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다 [출처: data/리비안_KR.txt]
- 테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다 [출처: data/테슬라_KR.txt]
- . 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다 [출처: data/테슬라_KR.txt]


## 4. 검색 성능 평가

### 4-1 테스트 데이터

- QA 데이터셋 합성
- 합성 데이터에 대한 검증 및 수정

`(1) 데이터 준비`

In [89]:
docs[0].metadata['source']

'data/리비안_KR.txt'

In [90]:
# meatadata에서 정보 추출 -> 문서 본문에 추가
os.path.split(docs[0].metadata['source'])[1].split('_')[0]

'리비안'

In [91]:
final_docs = []
for i, doc in enumerate(docs):
    new_doc = doc.copy()
    # metadata에 doc id를 추가
    new_doc.metadata['doc_id'] = i
    # 문장 구분 기호를 줄바꿈 문자로 변경
    new_doc.page_content = str(new_doc.page_content).replace("[.!?]\\s+", "\n")
    # metadata에서 정보를 추출하여 page_content에 추가
    corp_name = str(os.path.split(new_doc.metadata['source'])[1].split('_')[0])
    new_doc.page_content = f"{new_doc.page_content}\n\n(참고: 이 문서는 {corp_name}에 대한 정보를 담고 있습니다.)"
    final_docs.append(new_doc)


# 각 문서를 출력
for doc in final_docs[:3]:
    print(doc.page_content)
    print("-" * 50)
    print(doc.metadata)
    print("=" * 50)
    print()

.

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다

(참고: 이 문서는 리비안에 대한 정보를 담고 있습니다.)
--------------------------------------------------
{'source': 'data/리비안_KR.txt', 'doc_id': 0}

테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다

(참고: 이 문서는 테슬라에 대한 정보를 담고 있습니다.)
--------------------------------------------------
{'source': 'data/테슬라_KR.txt', 'doc_id': 1}

.

2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년에 비해 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 테슬라의 전 세계 누적 판매량은 4,962,975대를 초과했습니다. SMT Packaging에 따르면, 2023년 테슬라의 판매량은 전 세계 전기차 시장의 약 12.9%를 차지했습니다

(참고: 이 문서는 테슬라에 대한 정보를 담고 있습니다.)
--------------------------------------------------
{'source': 'data/테슬라_KR.txt', 'doc_id': 2}



In [92]:
# 문서를 저장
import json
with open('./data/final_docs_ver2.jsonl', 'wb') as f:
    for doc in final_docs:
        f.write(json.dumps(dict(doc)).encode('utf-8'))
        f.write(b'\n')

In [93]:
# JSONL 파일 로드하기
from langchain_community.document_loaders import JSONLoader

def metadata_func(record: dict, metadata: dict) -> dict:
    metadata = record.get("metadata")
    return metadata

json_loader = JSONLoader(
    file_path="./data/final_docs_ver2.jsonl",
    jq_schema=".",
    content_key="page_content",
    json_lines=True,
    metadata_func=metadata_func,
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
for doc in json_docs[:3]:
    print(doc.page_content)
    print("-" * 50)
    print(doc.metadata)
    print("=" * 50)
    print()

문서의 수: 8
--------------------------------------------------
.

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다

(참고: 이 문서는 리비안에 대한 정보를 담고 있습니다.)
--------------------------------------------------
{'source': 'data/리비안_KR.txt', 'doc_id': 0}

테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다

(참고: 이 문서는 테슬라에 대한 정보를 담고 있습니다.)
--------------------------------------------------
{'source': 'data/테슬라_KR.txt', 'doc_id': 1}

.

2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년에 비해 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 테슬라의 전 세계 누적 판매량은 4,962,975대를 초과했습니다. SMT Packaging에 따르면, 2023년 테슬라의 판매량은 전 세계 전기차 시장의 약 12.9%를 차지했습니다

(참고: 이 문서는 테슬라에 대한 정보를 담고 있습니다.)
--------------------------------------------------
{'source': 'data/테슬라_KR.txt', 'doc_id': 2}



In [94]:
# 데이터프레임으로 변환
import pandas as pd

test_data = []
for doc in json_docs:
    test_data.append({
        'context': str(doc.page_content),
        'source': str(doc.metadata.get('source', '')),
        'doc_id': str(doc.metadata.get('doc_id', '')),
    })

df_test = pd.DataFrame(test_data)
print(df_test.shape)
df_test.head()

(8, 3)


,context,source,doc_id
0,".\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌...",data/리비안_KR.txt,0
1,"테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차...",data/테슬라_KR.txt,1
2,".\n\n2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년에 비해...",data/테슬라_KR.txt,2
3,". 리비안은 디젤 하이브리드 버전, 브라질 원메이크 시리즈를 위한 R1 GT 레이싱...",data/리비안_KR.txt,3
4,. 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이...,data/테슬라_KR.txt,4


`(2) question - answer 합성`

In [95]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

# 질문-답변 쌍을 위한 Pydantic 모델 정의
class QAPair(BaseModel):
    """ 질문-답변 Pair"""
    question: str = Field(description="생성된 질문 (write your question in KOREAN)")
    answer: str = Field(description="질문에 대한 답변 (write the answer to the fact-based question in KOREAN, making sure it reflects the essence of the question)")

class QASet(BaseModel):
    qa_pairs: List[QAPair] = Field(description="질문-답변 Pair의 리스트")


# QA 생성 템플릿
QA_generation_template = """
Your task is to create {num_questions_per_chunk} fact-based question-answer pairs based on the provided context.
Each fact-based question should be answerable with a specific, concise piece of factual information from the context.
Formulate your questions in the style that users might use when asking a search engine.
Avoid including phrases like "according to the passage" or "based on the context" in your questions.
Ensure that the answer includes the essence of the question to provide clear and complete information.

---------------------------------------------------------
Provide your output in the following format:

{format_instructions}

---------------------------------------------------------
이제 컨텍스트를 제공합니다:

컨텍스트: {context}
"""

# ChatOpenAI 모델 초기화
qa_generator = ChatOpenAI(
    model="gpt-4o-mini",
    max_tokens=500,
    temperature=0.3,
)

# Pydantic 출력 파서 설정
pydantic_parser = PydanticOutputParser(pydantic_object=QASet)

# QA 생성 프롬프트 템플릿 생성
QA_generation_prompt = ChatPromptTemplate.from_template(
    template=QA_generation_template,
    partial_variables={"format_instructions": pydantic_parser.get_format_instructions()}
)

# QA 생성 체인 구성
QA_generate_chain = QA_generation_prompt | qa_generator | pydantic_parser

# 테스트 데이터셋 생성 함수
def generate_qa_dataset(context: str, num_questions: int) -> QASet:
    response = QA_generate_chain.invoke({
        "context": context,
        "num_questions_per_chunk": num_questions
    })
    return response


# QA 생성 테스트
test_context = df_test['context'][0]
print("컨텍스트:", test_context)
print()

qa_set = generate_qa_dataset(test_context, 2)
print("생성된 QA 쌍:")
for qa_pair in qa_set.qa_pairs:
    print(f"질문: {qa_pair.question}")
    print(f"답변: {qa_pair.answer}")
    print()

컨텍스트: .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다

(참고: 이 문서는 리비안에 대한 정보를 담고 있습니다.)

생성된 QA 쌍:
질문: 리비안의 초기 모델은 무엇인가요?
답변: 리비안의 초기 모델은 스포츠카 R1입니다.

질문: 스포츠카 R1의 디자인은 누구에 의해 이루어졌나요?
답변: 스포츠카 R1의 디자인은 피터 스티븐스에 의해 이루어졌습니다.



In [96]:
# 전체 문서에 대해 QA 생성 - Chunk 당 3개의 질문-답변 쌍 생성

NUM_QUESTIONS_PER_CHUNK = 3
outputs = []
for row in df_test.iterrows():

    qa_set = generate_qa_dataset(row[1]['context'], NUM_QUESTIONS_PER_CHUNK)

    for qa_pair in qa_set.qa_pairs:
        outputs.append({
            'context': [row[1]['context']],
            'source': [row[1]['source']],
            'doc_id': [row[1]['doc_id']],
            'question': qa_pair.question,
            'answer': qa_pair.answer
        })


df_qa_test = pd.DataFrame(outputs)
print(df_qa_test.shape)

df_qa_test.head()

(24, 5)


,context,source,doc_id,question,answer
0,"[.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 ...",[data/리비안_KR.txt],[0],리비안의 초기 모델은 무엇인가요?,리비안의 초기 모델은 스포츠카 R1입니다.
1,"[.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 ...",[data/리비안_KR.txt],[0],R1의 좌석 구성은 어떻게 되나요?,R1은 2+2 좌석 구성입니다.
2,"[.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 ...",[data/리비안_KR.txt],[0],R1은 어떤 구조를 특징으로 하나요?,R1은 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 합니다.
3,"[테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기...",[data/테슬라_KR.txt],[1],테슬라는 어디에 본사를 두고 있나요?,테슬라는 텍사스주 오스틴에 본사를 두고 있습니다.
4,"[테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기...",[data/테슬라_KR.txt],[1],테슬라는 언제 설립되었나요?,테슬라는 2003년에 설립되었습니다.


In [144]:
df_qa_test.to_excel("./data/qa_test.xlsx", index=False)

`(3) 테스트 데이터 검토 및 수정`



In [145]:
# Test 데이터셋에 대한 QA 생성 결과를 리뷰한 후 다시 로드
df_qa_test = pd.read_excel("./data/qa_test_revised.xlsx")
df_qa_test.shape

(24, 5)

In [146]:
df_qa_test.head()

,context,source,doc_id,question,answer
0,"['.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2...",['data/리비안_KR.txt'],['0'],리비안의 초기 모델은 무엇인가요?,리비안의 초기 모델은 스포츠카 R1입니다.
1,"['.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2...",['data/리비안_KR.txt'],['0'],R1의 좌석 구성은 어떻게 되나요?,R1은 2+2 좌석 구성입니다.
2,"['.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2...",['data/리비안_KR.txt'],['0'],R1은 어떤 구조를 특징으로 하나요?,R1은 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 합니다.
3,"['테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전...",['data/테슬라_KR.txt'],['1'],테슬라는 어디에 본사를 두고 있나요?,테슬라는 텍사스주 오스틴에 본사를 두고 있습니다.
4,"['테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전...",['data/테슬라_KR.txt'],['1'],테슬라는 언제 설립되었나요?,테슬라는 2003년에 설립되었습니다.


### 4-2 Information Retrieval 평가지표

- K-RAG 패키지 사용 (pip install krag)
- Hit Rate, MRR, mAP@k, NDCG@k 계산

#### 1) 평가도구 사용법

In [100]:
### 샘플 문서 데이터
from langchain_core.documents import Document

# 각 쿼리에 대한 정답 문서 
actual_docs = [
    # Query 1
    [
        Document(metadata={'id': 1}, page_content='Doc_1'),
    ],
    # Query 2
    [
        Document(metadata={'id': 2}, page_content='Doc_2'),
        Document(metadata={'id': 5}, page_content='Doc_5'),
    ],
]


# 각 쿼리에 대한 검색 결과 
predicted_docs = [
    # Query 1
    [
        Document(metadata={'id': 1}, page_content='Doc_1'),
        Document(metadata={'id': 5}, page_content='Doc_5'),
    ],

    # Query 2
    [
        Document(metadata={'id': 4}, page_content='Doc_4'),
        Document(metadata={'id': 1}, page_content='Doc_1'),
        Document(metadata={'id': 5}, page_content='Doc_5'),
        Document(metadata={'id': 2}, page_content='Doc_2'),
        Document(metadata={'id': 3}, page_content='Doc_3'),
    ],
]

In [101]:
# 섬플 데이터로 평가 (2개의 쿼리에 대한 검색 결과)
from krag.evaluators import OfflineRetrievalEvaluators

# 평가자 인스턴스 생성
evaluator = OfflineRetrievalEvaluators(
    actual_docs=actual_docs,
    predicted_docs=predicted_docs,    
)

`(1) Hit Rate`
- 각 쿼리에 대한 검색 문서 중에 실제 정답 문서가 포함되어 있으면 1, 그렇지 않으면 0으로 계산
- 전체 검색 쿼리에 대해서 평가를 진행하고 평균을 계산

In [102]:
# 계산 방법 (k=1)
(1 + 0) / 2

0.5

In [103]:
# 계산 방법 (k=2)
(1 + 0) / 2

0.5

In [104]:
# 계산 방법 (k=3)
(1 + 0) / 2

0.5

In [105]:
# 계산 방법 (k=4) -> 2번째 쿼리에 대해서 4개의 문서를 검색했을 때 정답이 모두 포함되어 있음
(1 + 1) / 2

1.0

In [106]:
# 'Hit Rate' 계산
k_values = [1, 2, 3, 4, 5]
for k in k_values:
    hit_rate = evaluator.calculate_hit_rate(k=k)
    print(f"Hit Rate @{k}: {hit_rate['hit_rate']:.3f}")

Hit Rate @1: 0.500
Hit Rate @2: 0.500
Hit Rate @3: 0.500
Hit Rate @4: 1.000
Hit Rate @5: 1.000


`(2) MRR (Mean Reciprocal Rank)`
- 각 쿼리에 대해서 관련 문서가 처음 반환된 순위의 역수를 계산한 후 그 평균을 구하는 방법
- MRR은 사용자가 원하는 결과를 얼마나 빨리 찾을 수 있는지를 평가 (순서를 고려)

In [107]:
# 계산 방법 (k=1)
(1/1 + 0/1) / 2

0.5

In [108]:
# 계산 방법 (k=3) -> 2번째 쿼리에 대해서 3번째 검색 문서가 관련 문서임 (첫 번째 정답만 고려) 
(1/1 + 1/3) / 2

0.6666666666666666

In [109]:
# 'Mean Reciprocal Rank' 계산
for k in k_values:
    mrr = evaluator.calculate_mrr(k=k)
    print(f"MRR @{k}: {mrr['mrr']:.3f}")

MRR @1: 0.500
MRR @2: 0.500
MRR @3: 0.667
MRR @4: 0.667
MRR @5: 0.667


`(3) mAP@k`
- mAP@k (Mean Average Precision at k)는 검색 결과의 평균 정밀도(AP, Average Precision)를 측정하는 지표
- 각 쿼리에 대한 평균 정밀도를 계산하고, 이들의 평균을 계산 (특히 k개의 상위 예측 문서 내에서 측정)
- 검색된 결과의 순서가 중요. 관련 문서가 상위에 있을수록 높은 정밀도를 획득
- 상위 k개의 결과에 집중하여 평가하므로, 사용자에게 우선적으로 보여지는 결과의 품질을 반영

- mAP@3 계산 과정:

   1. 쿼리 1에 대한 AP@3 계산:
      * 예측 결과: [Doc_1, Doc_5]
      * 실제 정답: [Doc_1]
      * Precision@1 = 1/1 = 1 (첫 번째 문서가 정답)
      * AP@3 = 1/1 = 1 (정답 문서가 첫 번째 위치에 있음)

   2. 쿼리 2에 대한 AP@3 계산:
      * 예측 결과: [Doc_4, Doc_1, Doc_5]
      * 실제 정답: [Doc_2, Doc_5]
      * Precision@1 = 0/1 = 0 (첫 번째 문서가 정답이 아님)
      * Precision@2 = 0/2 = 0 (두 번째 문서도 정답이 아님)
      * Precision@3 = 1/3 (세 번째 문서가 정답)
      * AP@3 = (0 + 0 + 1/3) / 2 = 1/6 ≈ 0.1667
      (정답 문서 중 하나만 상위 3개 안에 있고, 그 위치에서의 정밀도를 정답 문서 수로 나눔)

   3. mAP@3 계산:
      mAP@3 = (AP@3_쿼리1 + AP@3_쿼리2) / 2
            = (1 + 0.1667) / 2
            ≈ 0.5833
   
- 설명:
   - AP@k는 각 정답 문서의 위치(순서)에서의 정밀도를 합산한 후, 정답 문서의 총 개수로 나눕니다.
   - 쿼리 1의 경우, 유일한 정답 문서가 첫 번째 위치에 있어 완벽한 점수를 얻었습니다.
   - 쿼리 2의 경우, 두 개의 정답 문서 중 하나만 상위 3개 안에 포함되어 있어 낮은 점수를 받았습니다.
   - mAP@3은 이 두 쿼리의 AP@3 값의 평균입니다.


In [112]:
# mAP 계산
for k in k_values:
    map_score = evaluator.calculate_map(k=k)
    print(f"mAP@{k}: {map_score['map']:.3f}")

mAP@1: 0.500
mAP@2: 0.500
mAP@3: 0.583
mAP@4: 0.708
mAP@5: 0.708


`(4) NDCG`
- NDCG(Normalized Discounted Cumulative Gain)는 추천 시스템에서 결과의 순서가 얼마나 잘 맞는지를 평가하는 지표
- 각 문서의 순위에 따라 가중치를 부여하여 순위가 높은 관련 문서가 더 큰 영향을 주는 것으로 측정

- NDCG@3 계산 과정: 이진 관련성(0 또는 1)을 사용

   1. 쿼리 1:
      - 예측 결과: [Doc_1, Doc_5]
      - 실제 정답: [Doc_1]
      - 관련성 점수: [1, 0]
      - 이상적인 관련성 점수: [1, 0] (정답 문서가 하나만 있으므로)
      - DCG = (2^1 - 1) / log2(2) = 1
      - IDCG = (2^1 - 1) / log2(2) = 1
      - NDCG = 1 / 1 = 1

   2. 쿼리 2:
      - 예측 결과: [Doc_4, Doc_1, Doc_5]
      - 실제 정답: [Doc_2, Doc_5]
      - 관련성 점수: [0, 0, 1]
      - 이상적인 관련성 점수: [1, 0, 0] (정답 문서가 하나만 있으므로)
      - DCG = (2^0 - 1) / log2(2) + (2^0 - 1) / log2(3) + (2^1 - 1) / log2(4) = 0.5
      - IDCG = (2^1 - 1) / log2(2) + (2^0 - 1) / log2(3) + (2^0 - 1) / log2(4) = 1
      - NDCG = 0.5 / 1 = 0.5

   3. 평균 NDCG:
      (1 + 0.5) / 2 = 0.75

- 설명:
   - 쿼리 1에 대해서는 완벽한 순위를 보여줍니다 (NDCG = 1).
   - 쿼리 2에 대해서는 관련 문서가 상위 3개 안에 포함되어 있지만, 최적의 위치(첫 번째)는 아니라는 것을 반영 (NDCG = 0.5)
   - 0.75라는 최종 점수는 시스템이 전반적으로 양호한 성능을 보이고 있음을 나타냄. 완벽하지는 않지만, 관련 문서를 상위 순위에 배치하는 데 어느 정도 성공하고 있음을 의미

In [113]:
for k in k_values:
    ndcg = evaluator.calculate_ndcg(k=k)
    print(f"NDCG @{k}: {ndcg['ndcg']:.3f}")

NDCG @1: 0.500
NDCG @2: 0.500
NDCG @3: 0.750
NDCG @4: 0.785
NDCG @5: 0.785


- NDCG, mAP, MRR 비교
    - 목적:
        - NDCG는 전체 검색 결과의 순서와 관련성에 대한 정확도를 평가. 특히, 관련 문서가 상위에 위치할수록 높은 점수.
        - mAP는 상위 k개의 검색 결과에서 정밀도를 평가하여 관련 문서의 반환 정확성을 측정.
        - MRR은 첫 번째 관련 문서가 얼마나 빨리 나타나는지를 평가.

    - 특징:
        - NDCG는 순위에 따라 가중치를 부여하며, 순서에 따라 문서의 중요도가 감소하도록 설계.
        - mAP는 순서에 따른 가중치를 부여하지 않고, 각 쿼리의 정밀도 평균을 계산.
        - MRR은 첫 번째 관련 문서의 순위에 집중하며, 그 이후의 문서 순위는 고려하지 않음.

    - 적용 분야:
        - NDCG는 검색 결과의 전반적인 순서가 중요한 상황에 적합
        - mAP는 검색된 문서들이 얼마나 잘 맞는지를 종합적으로 평가할 때 사용
        - MRR은 사용자가 원하는 정보를 얼마나 빨리 찾는지를 평가할 때 유용

#### 2) 테스트 데이터셋 평가

In [163]:
df_qa_test.head(2)

,context,source,doc_id,question,answer
0,"['.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2...",['data/리비안_KR.txt'],['0'],리비안의 초기 모델은 무엇인가요?,리비안의 초기 모델은 스포츠카 R1입니다.
1,"['.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2...",['data/리비안_KR.txt'],['0'],R1의 좌석 구성은 어떻게 되나요?,R1은 2+2 좌석 구성입니다.


In [164]:
# 테스트 데이터셋의 특정 행에 있는 컨텍스트 데이터를 Document 객체 리스트로 변환
from langchain_core.documents import Document
from typing import List


def context_to_document(df_test: pd.DataFrame, idx: int) -> List[Document]:
    """
    테스트 데이터셋의 특정 행에 있는 컨텍스트 데이터를 Document 객체 리스트로 변환
    """

    context = eval(df_test['context'].iloc[idx])
    source = eval(df_test['source'].iloc[idx])
    doc_id = eval(df_test['doc_id'].iloc[idx])

    context_docs = []
    for c, s, d in zip(context, source, doc_id):
        doc = Document(page_content=c, metadata={'source': s, 'doc_id': d})
        context_docs.append(doc)

    return context_docs

#  0번 행의 테스트 데이터를 문서 객체로 변환
print("첫 번째 행의 컨텍스트 데이터:") 
print(df_qa_test['context'].iloc[0])
print("-" * 50)
print()

context_docs = context_to_document(df_qa_test, 0)
print("첫 번째 행의 컨텍스트 문서 객체:")
print(context_docs[0].page_content) 
print(context_docs[0].metadata)
print("=" * 50)
print()

첫 번째 행의 컨텍스트 데이터:
['.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다\n\n(참고: 이 문서는 리비안에 대한 정보를 담고 있습니다.)']
--------------------------------------------------

첫 번째 행의 컨텍스트 문서 객체:
.

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다

(참고: 이 문서는 리비안에 대한 정보를 담고 있습니다.)
{'source': 'data/리비안_KR.txt', 'doc_id': '0'}



`- Kiwi 토크나이저 + BM25 검색기`

In [165]:
from krag.tokenizers import KiwiTokenizer
from krag.retrievers import KiWiBM25RetrieverWithScore

# BM25 검색기 초기화 (k=5)
retriever_bm25_kiwi = KiWiBM25RetrieverWithScore(
    documents=final_docs, 
    kiwi_tokenizer=KiwiTokenizer(model_type='knlm', typos='basic'), 
    k=5, 
)       

In [166]:
# BM25 검색기를 사용하여 문서 검색
question = df_qa_test['question'].iloc[0]
print("질문:", question)
print("==============================================")
context = df_qa_test['context'].iloc[0]
print("관련 문서:", context)
print("==============================================")

# BM25 검색
retrieved_docs = retriever_bm25_kiwi.invoke(question)

# 검색 결과 출력 
for doc in retrieved_docs:
    print(doc.metadata["bm25_score"])    
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}, doc_id: {doc.metadata['doc_id']}]")
    print("------------------------------")

질문: 리비안의 초기 모델은 무엇인가요?
관련 문서: ['.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다\n\n(참고: 이 문서는 리비안에 대한 정보를 담고 있습니다.)']
4.418403726253867
- .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다

(참고: 이 문서는 리비안에 대한 정보를 담고 있습니다.) [출처: data/리비안_KR.txt, doc_id: 0]
------------------------------
3.4940325225324993
- 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다

(참고: 이 문서는 리비안에 대한 정보를 담고 있습니다.) [출처: data/리비안_KR.txt, doc_id: 5]
------------------------------
1.551364854060795
- . 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다

(참고: 이 문서는 테슬라에 대한 정보를 담고 있습니다.) 

In [167]:
# 전체 테스트 데이터셋에 대하여 평가지표 계산
from langchain_core.retrievers import BaseRetriever

def evaluate_qa_test(df_qa_test: pd.DataFrame, retriever: BaseRetriever, k=2) -> dict:
    """
    테스트 데이터셋에 대한 검색 결과 평가
    """

    context_docs = []
    retrieved_docs = []

    df_test = df_qa_test.copy()
    
    for idx, _ in df_test.iterrows():
        question = df_test['question'].iloc[idx]
        context_doc = context_to_document(df_test, idx)
        context_docs.append(context_doc)
        retrieved_doc = retriever.invoke(question)  
        retrieved_docs.append(retrieved_doc)  


    # 평가자 인스턴스 생성
    evaluator = OfflineRetrievalEvaluators(
        actual_docs=context_docs,
        predicted_docs=retrieved_docs,      
    )


    # 평가지표 계산
    hit_rate = evaluator.calculate_hit_rate(k=k)['hit_rate']
    mrr = evaluator.calculate_mrr(k=k)['mrr']
    map_score = evaluator.calculate_map(k=k)['map']
    ndcg = evaluator.calculate_ndcg(k=k)['ndcg']

    print(f"K={k}")
    print("------------------------------------------------------------")
    print(f"Hit Rate: {hit_rate:.3f}")
    print(f"MRR: {mrr:.3f}")
    print(f"MAP: {map_score:.3f}")
    print(f"NDCG: {ndcg:.3f}")
    print("============================================================")
    print()

    result = {
        'hit_rate': hit_rate,
        'mrr': mrr,
        'map': map_score,
        'ndcg': ndcg,
        
    }

    return pd.Series(result)

In [168]:
# 평가 (k=1)
retriever_bm25_kiwi.k = 1
result_bm25_k1 = evaluate_qa_test(df_qa_test, retriever_bm25_kiwi, k=1)

K=1
------------------------------------------------------------
Hit Rate: 0.875
MRR: 0.875
MAP: 0.875
NDCG: 0.875



In [169]:
# 평가 (k=2)
retriever_bm25_kiwi.k = 2
result_bm25_k2 = evaluate_qa_test(df_qa_test, retriever_bm25_kiwi, k=2)

K=2
------------------------------------------------------------
Hit Rate: 0.875
MRR: 0.875
MAP: 0.875
NDCG: 0.875



`- Chroma 벡터저장소 검색기`

In [178]:
# 벡터스토어 로드
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings


embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")


chroma_db = Chroma.from_documents(
    documents=final_docs,
    embedding=embeddings_model,
    collection_name="hf_bge_m3",
    persist_directory="./chroma_db",
)

# Chroma 검색기 초기화
retriever_chroma_hf = chroma_db.as_retriever(
    search_kwargs={"k": 5},
)

In [179]:
# 평가 (k=1)
result_chroma_hf_k1 = evaluate_qa_test(df_qa_test, retriever_chroma_hf, k=1)

K=1
------------------------------------------------------------
Hit Rate: 0.792
MRR: 0.792
MAP: 0.792
NDCG: 0.792



In [180]:
# 평가 (k=2)
result_chroma_hf_k2 = evaluate_qa_test(df_qa_test, retriever_chroma_hf, k=2)

K=2
------------------------------------------------------------
Hit Rate: 0.875
MRR: 0.833
MAP: 0.833
NDCG: 0.844



In [181]:
# 평가 (k=3)
result_chroma_hf_k3 = evaluate_qa_test(df_qa_test, retriever_chroma_hf, k=3)

K=3
------------------------------------------------------------
Hit Rate: 0.875
MRR: 0.833
MAP: 0.833
NDCG: 0.844



In [174]:
from langchain.retrievers import EnsembleRetriever

retriever_bm25_kiwi.k = 5
ensemble_retrievers = [retriever_chroma_hf, retriever_bm25_kiwi]
ensemble_retriever = EnsembleRetriever(
    retrievers=ensemble_retrievers, 
    weights=[0.5, 0.5]
)

# 평가 (k=1)
result_ensemble_k1 = evaluate_qa_test(df_qa_test, ensemble_retriever, k=1)

K=1
------------------------------------------------------------
Hit Rate: 0.833
MRR: 0.833
MAP: 0.833
NDCG: 0.833



In [175]:
# 평가 (k=2)
result_ensemble_k2 = evaluate_qa_test(df_qa_test, ensemble_retriever, k=2)

K=2
------------------------------------------------------------
Hit Rate: 0.875
MRR: 0.854
MAP: 0.854
NDCG: 0.860

